# Projects markdown generator for academicpages

Takes a TSV of projects with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, slides_url, video_url, excerpt, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat projects.json

[
    {
        "title": "The Sound of Suffrage - Modeling Gender and Power in Parliamentary Speech",
        "contributors": "Mandira Sawkar, Omar Khursheed, Dr. Ashique KhudaBukhsh",
        "type": "Academic",
        "project_date": "2025-08-01 - 2026-05-01", 
        "url_slug": "The-Sound-of-Suffrage-Modeling-Gender-and-Power-in-Parliamentary-Speech", 
        "teaser_url": "",
        "report_url": "",
        "code_url": "",
        "excerpt": "Developing NLP pipeline over 200 years of Hansard debates (1803–2005) using LLMs to study gender bias, framing, and speaker dynamics. Applying representation analysis and framing detection to uncover temporal shifts in discourse, generating insights for bias and fairness in AI systems.",
        "my_contribution": ""
    },
    {
        "title": "Detecting Deception: Intelligent Systems for Fighting Misinformation",
        "contributors": "Mandira Sawkar, Leona Joseph, Vivek Senthil",
        "type": "Academic",
        "project_date":

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
projects = pd.read_json("projects.json")
projects

,title,contributors,type,project_date,url_slug,teaser_url,report_url,code_url,excerpt,my_contribution
0,The Sound of Suffrage - Modeling Gender and Po...,"Mandira Sawkar, Omar Khursheed, Dr. Ashique Kh...",Academic,2025-08-01 - 2026-05-01,The-Sound-of-Suffrage-Modeling-Gender-and-Powe...,,,,Developing NLP pipeline over 200 years of Hans...,
1,Detecting Deception: Intelligent Systems for F...,"Mandira Sawkar, Leona Joseph, Vivek Senthil",Academic,2025-05-01,Intelligent-Systems-for-Fighting-Misinformation,,,https://github.com/mandira-sawkar/Intelligent-...,Designed a logic-gated fact-checking system in...,Conducted extensive literature review and deve...
2,Beyond Hallucinations: A Multi-Agent Approach ...,Mandira Sawkar,Academic,2025-05-05,Beyond-Hallucinations-Multi-Agent-Fact-Checking,,,https://github.com/mandira-sawkar/Beyond-Hallu...,Developed a theoretical framework consisting o...,Sole contributor.
3,AI Alignment in Criminal Justice - Fair Jury S...,"Mandira Sawkar, Leona Joseph, Shrid Pant",Academic,2024-12-04,AI-Alignment-in-Criminal-Justice,,,https://github.com/mandira-sawkar/AI-Alignment...,Created a theoretical hybrid model for fair ju...,Conducted literature review and helped develop...
4,LLM-as-a-SupremeCourt-Judge,Mandira Sawkar,Academic,2024-12-08,LLM-as-a-SupremeCourt-Judge,,,https://github.com/mandira-sawkar/LLM-as-a-Sup...,"Developed an AI framework with LLMs (GPT API, ...",Sole contributor.
5,Interpretable Parking Ticket Location Classifier,Mandira Sawkar,Academic,2023-01-01,interpretable-parking-ticket-location-classifier,NaN,NaN,https://github.com/mandira-sawkar/Interpretabl...,Built a SparkML RandomForest classifier to pre...,Sole contributor.
6,Product Shipping Status Classifier,Mandira Sawkar,Academic,2022-12-01,product-shipping-status-classifier,NaN,NaN,https://github.com/mandira-sawkar/Product-Ship...,Created a RandomForest model to predict issues...,Sole contributor.
7,Cancer Detection as a Microservice,Mandira Sawkar,Academic,2022-12-01,cancer-detection-as-a-microservice,NaN,NaN,https://github.com/mandira-sawkar/Cancer-Detec...,Trained a Voting Classifier to predict cancer ...,Sole contributor.
8,getPaid,Mandira Sawkar,Academic,2022-11-01,get-paid,,,https://github.com/mandira-sawkar/getPaid,Created a SparkML RandomForest model to predic...,Sole contributor.


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in projects.iterrows():
    
    md_filename = str(item.project_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.project_date) + "-" + item.url_slug 
    year = item.project_date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: projects" + "\n"
    md += f'urlslug: "{item.url_slug }"\n'
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Project"\n'
    md += "permalink: /projects/" + html_filename + "\n"
    if len(str(item.contributors)) > 3:
        md += 'contributors: "' + item.contributors + '"\n'
    if len(str(item.my_contribution)) > 3:
        md += 'contribution: "' + item.my_contribution + '"\n'
    md += "date: " + str(item.project_date) + "\n"
    if len(str(item.teaser_url)) > 5:
        md += "teaserurl: '" + item.teaser_url + "'\n"
    report_url = None
    if len(str(item.report_url)) > 5:
        report_url = str(item.report_url)
        md += "reporturl: '" + item.report_url + "'\n"
    code_url = None
    if len(str(item.code_url)) > 5:
        code_url = str(item.code_url)
        md += "codeurl: '" + item.code_url + "'\n"
    if len(str(item.excerpt)) > 5:
        md += "excerpt: '" + html_escape(item.excerpt) + "'\n"
           
    md += "---\n"

    more_info = []
    if code_url is not None:
        more_info.append(f'[[Code]({code_url})]')
    if report_url is not None:
        more_info.append(f'[[Technical report]({report_url})]')
    if len(item.contributors) > 0:
        md += f"\n{item.contributors}\n"
    if len(str(item.excerpt)) > 4:
        md += f"\n**Description:**\n{html_escape(item.excerpt)}\n"
    if len(str(item.my_contribution)) > 4:
        md += f"\n**My contribution:**\n{html_escape(item.my_contribution)}\n"
    if len(more_info) > 0:
        md += f"\n**Resources:** {' '.join(more_info)}\n"
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_projects/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_projects

2022-11-01-get-paid.md
2022-12-01-cancer-detection-as-a-microservice.md
2022-12-01-product-shipping-status-classifier.md
2023-01-01-interpretable-parking-ticket-location-classifier.md
2024-12-04-AI-Alignment-in-Criminal-Justice.md
2024-12-08-LLM-as-a-SupremeCourt-Judge.md
2025-05-01-Intelligent-Systems-for-Fighting-Misinformation.md
2025-05-05-Beyond-Hallucinations-Multi-Agent-Fact-Checking.md
2025-08-01 - 2026-05-01-The-Sound-of-Suffrage-Modeling-Gender-and-Power-in-Parliamentary-Speech.md


In [7]:
!cat ../_projects/2020-12-14-ut-mscs-deep-learning-project.md

cat: ../_projects/2020-12-14-ut-mscs-deep-learning-project.md: No such file or directory
